In [3]:
import json
from collections import Counter, defaultdict
from wiki_labels import qid_lab_get, qid_en_desc_get
from wiki_location import q2cc
from text_utils import cl
import sys
sys.path.insert(0, '/projekti/mondoAPI')
from pnu.parse import parse
from api.db import db
import pandas as pd
import gzip

sys.path.insert(0, '/projekti/nelma')
sys.path.insert(0, '/projekti/nelma/mondoDB')

from mondoDB.parse_thai_name import norm_thai, tokenise, parse_thai

from tqdm.notebook import tqdm
from wikilang2iso import wikil2cc, cc2lang, iso2w, w2iso

print(qid_lab_get(42, 'en').keys(), parse('davor lauc')['tags'][0], parse('반기문')['tags'][0])
wikil2cc['Q1860']['US'], cc2lang['CH'], cc2lang['US']

/projekti/nelma/mondoDB/parse_thai_name.py:23: SyntaxWarning: invalid escape sequence '\.'
  title_male = re.compile('^นาย|เด็กชาย|ด\.ช\.|สยา')
/projekti/nelma/mondoDB/parse_thai_name.py:24: SyntaxWarning: invalid escape sequence '\.'
  title_female = re.compile('^นาง|นางสาว|เด็กหญิง|ด\.ญ\.|น\.ส\.')
/projekti/nelma/mondoDB/parse_thai_name.py:27: SyntaxWarning: invalid escape sequence '\.'
  title_company = re.compile('บริษัท|ห้างหุ้นส่วนจำกัด|หจก\.|ร้าน')


dict_keys(['douglas adams']) {'tokens': ['davor', 'lauc'], 'labels': ('fn1_hr_Latn_HR', 'ln_hr_Latn_HR'), 'prob': 92.0626} {'tokens': ['반', '기문'], 'labels': ('ln_ko_Hang_KR', 'fn1_ko_Hang_KR'), 'prob': 97.0899}


(94, ['it', 'fr', 'de', 'en'], ['en'])

In [47]:
from wikilang2iso import get_wiki_cc, cc_weights

for i, l in tqdm(enumerate(gzip.open('/backup/wikidata/wikinelma.jsonl.gz')), total=26_074_182):
    j = json.loads(l)
    if j['type'] == 'per':
        qid = int(j['wiki_id'][1:])
        cc, ccs = get_wiki_cc({k: v for k, v in j in k in cc_weights})
        break 
ccs

  0%|          | 0/17630406 [00:00<?, ?it/s]

[('US', 15.4)]

In [4]:
notable = pd.read_csv('/backup/wikidata/cross-verified-database.csv.gz', compression='gzip', encoding='latin-1')
pd.set_option('display.max_columns', None)
qrank = {v['wikidata_code']:v['ranking_visib_5criteria'] for _, v in notable.iterrows()}
notable

KeyboardInterrupt: 

In [ ]:
fo = gzip.open('/backup/wikidata/wiki_person.tsv.gz', 'wt')
fo.write('qid\tname\tfn\tln\tgender\tcc_first\tnative_lang\tccs\tdesc_en\tdesc_nl\tdob\tdod\timage\tsort\n')
for i, l in tqdm(enumerate(gzip.open('/backup/wikidata/wikinelma.jsonl.gz')), total=25_042_218):
    j = json.loads(l)
    if j['type'] == 'per':
        qid = int(j['wiki_id'][1:])
        cc, ccs = get_wiki_cc({k: v for k, v in j if k in cc_weights})
        name = None
        if j['native_language']:
            langs = Counter([iso2w[q[5:]] for q in j['native_language'] if q[5:] in iso2w])
        else:
            langs = Counter()
        langs.update(cc2lang[cc])
        native_lang = langs.most_common()[0][0] if langs else ''
        if '-' in native_lang:
            native_lang = native_lang.split('-')[0]
        elif len(native_lang)>2:
            native_lang = native_lang[:2]
        if j['name_native']:
            name = cl(j['name_native'][0]).lower()
        else:
            for l, f in langs.most_common():
                name = qid_lab_get(qid, l)
                if name:
                    name = list(name)[0]
                    break
        if not name:
            try:
                name = list(qid_lab_get(qid, 'en', include_alt=False).keys())[0]
            except:
                try:
                    name = list(j['l'].values())[0][0]
                except:
                    name = ''
        if name:
            namel = name.lower().strip()
            desc = qid_en_desc_get(qid)
            dob = j['dob'][0][:4] if j['dob'] else ''
            dod = j['dod'][0][:4] if 'dod' in j and j['dod'] else ''
            
            image = j['picture'][0] if j['picture'] else ''
            sort = qrank[j['wiki_id']] if  j['wiki_id'] in qrank  else (100_000_000+(int(dob) if dob else 1980)-(50 if image else 0)-(10 if desc else 0))

            fn = ''
            if j['fn']:
                fns = qid_lab_get(int(j['fn'][0][6:]), include_alt=True).keys()
                fns = Counter({a:len(a) for a in fns if a in namel})
                if len(fns)>0:
                    fn = fns.most_common()[0][0]

            ln = ''
            if j['ln']:
                fns = qid_lab_get(int(j['ln'][0][6:]), include_alt=True).keys()
                fns = Counter({a:len(a) for a in fns if a in namel})
                if len(fns)>0:
                    ln = fns.most_common()[0][0]
            # only 
            gender = ''
            if j['gender']==['WIKI_Q6581097']:
                gender = 'm'
            elif j['gender']==['WIKI_Q6581072']:
                gender = 'f'
            
            if False and not fn or not ln:
                if ',' in namel:
                    if ' ' in namel.split(',')[0].strip():
                        namel = namel.split(',')[0].strip()
                    else:
                        namel = namel.split(',')[1].strip()+' '+namel.split(',')[0].strip()
                    #print(name, namel)
                parts = namel.split(' ')
                if len(parts) == 1 and fn:
                    if namel.startswith(fn):
                        parts = [fn, namel[len(fn):]]
                    elif namel.endswith(fn):
                        parts = [fn, namel[:len(namel)-len(fn)]]
                if len(parts) == 1 and ln:
                    if namel.startswith(ln):
                        parts = [namel[len(ln):], ln]
                    elif namel.endswith(ln):
                        parts = [namel[:len(namel)-len(ln)], ln]

                if len(parts) == 2:
                    if fn and parts[0]==fn:
                        ln = parts[1]
                    elif fn and parts[1]==fn:
                        ln = parts[0]
                    elif ln and parts[1]==ln:
                        fn = parts[0]
                    elif ln and parts[0]==ln:
                        fn = parts[1]
                    else:
                        fn, ln = parts
                elif len(parts) > 2:
                    fn, ln = parts[0], parts[-1]
                    
            if not cc:
                cc = ''

            desc_en = qid_en_desc_get(qid)
            for q in j['positions']:
                desc_nl = list(qid_lab_get(int(q[6:]), native_lang, include_alt=False).keys())
                desc_nl = desc_nl[0] if desc_nl else ''

                if desc_nl:
                    break


            fo.write(f'{qid}\t{name}\t{fn}\t{ln}\t{gender}\t{cc}\t{native_lang}\t{ccs}\t{desc_en}\t{desc_nl}\t{dob}\t{dod}\t{image}\t{sort}\n')
    #if i>100:
    #    break
fo.close()
    

In [ ]:
fo = open('/backup/wikidata/wiki_parsed.tsv', 'w')
fi = open('/backup/wikidata/wiki_person.tsv', 'r')
fi.readline()
fo.write(f"qid\tname\tprob\ttitle\tfn\tln\tfn_org\tln_org\n") 

for i, l in tqdm(enumerate(fi), total=9928121 ):
    qid, name, fn_org, ln_org, gender, desc, cc, dob, image, sort = l.strip('\n').split('\t')
    rec = parse(name)
    titles, fn, ln = [], [], []
    prob = -1 
    if rec['tags'] and rec['tags'][0]['prob']>0.2:
        prob = rec['tags'][0]['prob']
        for tok, typ in zip(rec['tags'][0]['tokens'], rec['tags'][0]['labels']):
            if typ[:5] == 'title':
                titles.append(tok)
            elif typ[:2] == 'fn':
                fn.append(tok)
            elif typ[:2] == 'ln':
                ln.append(tok)
    fo.write(f"{qid}\t{name}\t{prob}\t{','.join(titles)}\t{','.join(fn)}\t{','.join(ln)}\t{fn_org}\t{ln_org}\n") 
fo.close()

In [ ]:
!wc -l /backup/wikidata/wiki_person.tsv

In [ ]:
uk = 0
fn = '/projekti/mondonomo/books/thai/thai_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('thai_', 'nonthai_'), 'w')
for i, l in enumerate(open('/backup/wikidata/wiki_person.jsonl')):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'th' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["th"][0].replace(',',' ')
        if cc == 'TH':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head /projekti/mondonomo/books/thai/thai_wiki.csv

In [ ]:
from tqdm.notebook import tqdm
uk = 0
fn = '/backup/wikidata/hr_wiki.csv'
fo = open(fn, 'w')
fo2 = open(fn.replace('hr_', 'nonhr_'), 'w')
for l in tqdm(open('/backup/wikidata/wiki_person.jsonl'), total=10_044_571):
    j = json.loads(l)
    cc = get_wiki_cc(j['country'], j['birth_place'], j['language'])
    if 'en' in j['l'] and 'hr' in j['l'] and cc:
        native = j["name_native"][0] if j["name_native"] else ''
        en = j["l"]["en"][0].replace(',',' ')
        th = j["l"]["hr"][0].replace(',',' ')
        if cc == 'HR':
            fo.write(f'{j["id"]},{cc},{en},{th}\n')
        else:
            fo2.write(f'{j["id"]},{cc},{en},{th}\n')
        uk += 1
fo.close()
!head $fn

In [ ]:
parse('George Washington')